In [ ]:
import ctypes
from ctypes import c_char_p
import os
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from sklearn.model_selection import train_test_split
from tqdm.notebook import tqdm

In [ ]:

def load_images_from_folder(folder, label):
    images = []
    labels = []
    for filename in os.listdir(folder):
        img_path = os.path.join(folder, filename)
        img = Image.open(img_path).convert('L')  # Conversion en gris
        img = Image.open(img_path).convert('RGB')  # Conversion en gris
        img = img.resize((128, 128))  # Resize les images
        img_array = np.array(img, dtype=np.float64).flatten()  # Flatten les vecteurs
        images.append(img_array)
        labels.append(label)
    return images, labels

folder_a = 'data/dataset/resized/gas_giant'
folder_b = 'data/dataset/resized/neptune-like'
folder_c = 'data/dataset/resized/super_earth'

images_a, labels_a = load_images_from_folder(folder_a, [1.0, 0.0, 0.0])
images_b, labels_b = load_images_from_folder(folder_b, [0.0, 1.0, 0.0])
images_c, labels_c = load_images_from_folder(folder_c, [0.0, 0.0, 1.0])

X = np.array(images_a + images_b + images_c)
y = np.array(labels_a + labels_b + labels_c)

# Séparer les données entrainement / test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Training data shape: {X_train.shape}")
print(f"Training data shape: {y_train.shape}")
print(f"Test data shape: {X_test.shape}")
print(f"Test data shape: {y_test.shape}")

In [ ]:
def display_selected_images(images, labels, indices, nrows, ncols):
    fig, axes = plt.subplots(nrows, ncols, figsize=(10, 10))
    axes = axes.flatten()
    for i, ax in enumerate(axes):
        if i < len(indices):
            index = indices[i]
            # image_2d = images[index].reshape((64, 64)) 
            image_3d = images[index].reshape((128, 128, 3)) 
            if image_3d.max() > 1.0:
                image_3d = image_3d / 255.0 
            # ax.imshow(image_2d, cmap='gray')
            ax.imshow(image_3d)
            ax.set_title(f"Label: {labels[index]}")
        ax.axis('off')
    plt.tight_layout()
    plt.show()

In [ ]:
selected_indices = [39, 100, 70, 320, 230, 159]
nrows, ncols = 2, 3

display_selected_images(X_train, y_train, selected_indices, nrows, ncols)

# Perceptron multi couche

In [ ]:
mlp_model = ctypes.CDLL("modele/mlp/target/release/libmlp_classification.so")

# Définir les types d'arguments et de retour des fonctions C
PROGRESS_CALLBACK = ctypes.CFUNCTYPE(None, ctypes.c_int, ctypes.c_double)
mlp_model.mlpInit.argtypes = [ctypes.POINTER(ctypes.c_uint), ctypes.c_size_t, ctypes.c_double]
mlp_model.mlpInit.restype = ctypes.POINTER(ctypes.c_void_p)

mlp_model.mlpTrain.argtypes = [
    ctypes.POINTER(ctypes.c_void_p),
    ctypes.POINTER(ctypes.c_double),
    ctypes.POINTER(ctypes.c_double),
    ctypes.c_size_t,
    ctypes.c_size_t,
    ctypes.c_size_t,
    ctypes.c_size_t,
    ctypes.c_size_t,
    ctypes.c_bool,
    PROGRESS_CALLBACK,
    ctypes.c_size_t
]
mlp_model.mlpTrain.restype = ctypes.c_double


mlp_model.mlpPredict.argtypes = [
    ctypes.POINTER(ctypes.c_void_p),
    ctypes.POINTER(ctypes.c_double),
    ctypes.c_size_t,
    ctypes.c_bool,
    ctypes.POINTER(ctypes.c_double)
]
mlp_model.mlpPredict.restype = None

mlp_model.mlpFree.argtypes = [ctypes.POINTER(ctypes.c_void_p)]
mlp_model.mlpFree.restype = None

In [ ]:
print(f"vecteur : {X_train}")
print(f"type : {type(X_train[0][0])}")
print(f"pointer : {X_train.ctypes.data_as(ctypes.POINTER(ctypes.c_double))}")
print(f"Training data shape: {X_train.shape}")
print(f"Training data shape: {y_train.shape}")
print(f"Test data shape: {X_test.shape}")
print(f"Test data shape: {y_test.shape}")

In [ ]:
neurons_size = np.array([X_train.shape[1], 32, 3], dtype=np.uintp)
learning_rate = 0.001
epochs = 1000
callback_interval = 10

parameter_ptr = neurons_size.ctypes.data_as(ctypes.POINTER(ctypes.c_uint))
nn = mlp_model.mlpInit(parameter_ptr, neurons_size.size, learning_rate)

In [ ]:
progress_bar = tqdm(total=epochs, desc="Training Progress")

def progress_callback(epochs, loss):
    progress_bar.update(callback_interval)
    progress_bar.set_postfix(loss=loss)

callback_func = PROGRESS_CALLBACK(progress_callback)

loss = mlp_model.mlpTrain(
    nn,
    X_train.ctypes.data_as(ctypes.POINTER(ctypes.c_double)),
    y_train.ctypes.data_as(ctypes.POINTER(ctypes.c_double)),
    X_train.shape[0],
    X_train.shape[1],
    y_train.shape[1],
    epochs,
    32,
    True,  
    callback_func,
    callback_interval  
)
progress_bar.close()

In [ ]:
predictions_list = []
labels_list = []

for k in range(len(X_test)):
    predictions = np.zeros(y_train.shape[1], dtype=np.float64)
    mlp_model.mlpPredict(
        nn,
        X_test[k].ctypes.data_as(ctypes.POINTER(ctypes.c_double)),
        X_test[k].size,
        True,
        predictions.ctypes.data_as(ctypes.POINTER(ctypes.c_double))
    )
    predictions_list.append(predictions)
    labels_list.append(y_test[k])

In [ ]:
predictions_array = np.array(predictions_list)
labels_array = np.array(labels_list)

for i in range(10):
    print(f"Sample {i}, True Label: {labels_array[i]}, Predictions: {predictions_array[i]}")

accuracy = np.mean(np.argmax(predictions_array, axis=1) == np.argmax(labels_array, axis=1))
print(f"Accuracy: {accuracy * 100:.2f}%")

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

cm = confusion_matrix(np.argmax(labels_array, axis=1), np.argmax(predictions_array, axis=1))
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot(cmap=plt.cm.Blues)
plt.show()

In [ ]:
mlp_model.mlpFree(nn)